In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('../data/energy_consumption_preprocessed.csv')

In [3]:
df.head()

,Temperature,Humidity,SquareFootage,Occupancy,RenewableEnergy,Season_Spring,Season_Summer,Season_Winter,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night,Holiday_Yes,HVACUsage_On,LightingUsage_On,EnergyConsumption
0,25.139433,43.431581,1565.693999,5,2.774699,False,False,True,False,False,True,False,True,False,75.364373
1,27.731651,54.225919,1411.064918,1,21.831384,False,False,True,False,False,True,False,True,True,83.401855
2,28.704277,58.907658,1755.715009,2,6.764672,False,False,True,False,False,True,False,False,False,78.270888
3,20.080469,50.371637,1452.316318,1,8.623447,False,False,True,False,False,True,False,False,True,56.519850
4,23.097359,51.401421,1094.130359,9,3.071969,False,False,True,False,False,True,False,True,False,70.811732


In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

X = df.drop('EnergyConsumption', axis=1)
y = df['EnergyConsumption']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(X_test.shape)

# print scaled data
print(X_train[:5])


(4000, 14)
(1000, 14)
[[0.54759105 0.75537573 0.08914184 0.55555556 0.26446175 0.
  0.         0.         0.         1.         0.         1.
  0.         0.        ]
 [0.25686188 0.84628361 0.44550102 0.44444444 0.47425702 0.
  0.         1.         0.         0.         0.         1.
  1.         1.        ]
 [0.5563993  0.16976924 0.53558351 0.88888889 0.12826703 0.
  0.         1.         0.         1.         0.         0.
  1.         0.        ]
 [0.02005537 0.69851482 0.38369588 0.44444444 0.38363405 0.
  0.         0.         1.         0.         0.         1.
  0.         0.        ]
 [0.00270167 0.83355062 0.9334812  0.66666667 0.55751224 0.
  0.         0.         0.         0.         0.         1.
  0.         0.        ]]


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
# LSTM
from tensorflow.keras.layers import LSTM

In [7]:
model = Sequential([
    # LSTM layer to capture temporal dependencies
    LSTM(64, input_shape=(None, X_train.shape[1]), return_sequences=True),
    Dropout(0.2),
    
    # Additional LSTM layer
    LSTM(32),
    Dropout(0.1),
    
    # Dense layers for temporal difference approximation
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    
    # Output layer for energy consumption prediction
    Dense(1)
])

2025-03-25 22:17:56.766678: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/cristiano/anaconda3/envs/env_ds/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 64)       │        20,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,865 (144.00 KB)

 Trainable params: 36,865 (144.00 KB)

 Non-trainable params: 0 (0.00 B)